In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import os 

In [3]:
movie = pd.read_csv('D:\\project_movie\data\\tmdb_5000_movies.csv')
credit = pd.read_csv('D:\\project_movie\data\\tmdb_5000_credits.csv')

In [4]:
movie.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [5]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [6]:
credit.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [11]:
movie = movie.merge(credit, on ="title" )

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import streamlit as st
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier

# تحميل البيانات
def load_data():
    df = pd.read_csv("/mnt/data/expanded_tourism_program_data.csv")
    return df

# تنظيف البيانات
def clean_data(df):
    cols = [col for col in df.columns if df[col].nunique() == 2 and df[col].dtype == 'object']
    for col in cols:
        df[col] = df[col].map({'Yes': 1, 'No': 0})
    encoder = LabelEncoder()
    df['Program_Recommended'] = encoder.fit_transform(df['Program_Recommended'])
    return df, cols, encoder

# تحليل البيانات
def analyze_data(df):
    st.subheader("📊 تحليل العلاقات بين الميزات")
    fig, ax = plt.subplots(figsize=(10, 5))
    sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt='.2f', ax=ax)
    st.pyplot(fig)

# تدريب الموديل
def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
    model.fit(X_train, y_train)
    return model, X

# توقع البرنامج السياحي
def predict_program(model, X, encoder, user_input):
    user_df = pd.DataFrame([user_input], columns=X.columns)
    prediction = model.predict(user_df)
    return encoder.inverse_transform(prediction)[0]

# تنفيذ الكود مع Streamlit
st.title("🚀 نظام توصية البرامج السياحية")
df = load_data()
df, cols, encoder = clean_data(df)
analyze_data(df)
model, X = train_model(df[cols], df['Program_Recommended'])

# إدخال بيانات المستخدم في Streamlit
st.subheader("📝 أدخل تفضيلاتك السياحية")
user_input = []
for col in cols:
    user_input.append(st.radio(f"{col}", [1, 0], format_func=lambda x: "نعم" if x == 1 else "لا"))

if st.button("🔍 احصل على التوصية"):
    recommendation = predict_program(model, X, encoder, user_input)
    st.success(f"✨ البرنامج السياحي المناسب لك: {recommendation}")


ModuleNotFoundError: No module named 'seaborn'

In [1]:
import pandas as pd

In [2]:
import pandas as pd
import random

# النموذج الأساسي (20 سطر)
data = [
    ["Salah El-Din Citadel", "Cairo", "5 AM - 8 PM", 60, "https://www.google.com/maps/search/?api=1&query=Salah+El-Din+Citadel", "A medieval Islamic-era fortress with mosques and museums.", "fortress|mosque|museum|medieval|cairo", "Historical"],
    ["Baron Palace", "Cairo", "9 AM - 5 PM", 60, "https://www.google.com/maps/search/?api=1&query=Baron+Palace", "A unique Hindu-style palace built in the early 20th century.", "palace|hindu|architecture|20th-century|cairo", "Cultural"],
    # أضف بقية الأسطر الـ 18 من النموذج اللي فوق هنا
]

# قوائم لتوليد بيانات عشوائية
governorates = ["Cairo", "Giza", "Luxor", "Aswan", "Alexandria", "Fayoum", "Minya", "Sohag", "Sinai"]
hours = ["5 AM - 8 PM", "9 AM - 5 PM", "7 AM - 5 PM", "8 AM - 5 PM", "6 AM - 9 PM"]
prices = [6, 36, 60, 100]
types = ["Historical", "Cultural", "Religious"]
tags = ["temple", "museum", "pyramid", "ancient", "fortress", "mosque", "church", "tombs", "pharaohs"]

# توليد 50,000 صف
generated_data = []
for i in range(50000):
    site_name = f"Site {i+1}"
    governorate = random.choice(governorates)
    visiting_hours = random.choice(hours)
    price = random.choice(prices)
    maps_link = f"https://www.google.com/maps/search/?api=1&query=Site+{i+1}"
    description = f"A unique site in {governorate} with historical significance."
    site_tags = "|".join(random.sample(tags, 3)) + f"|{governorate.lower()}"
    tourism_type = random.choice(types)
    generated_data.append([site_name, governorate, visiting_hours, price, maps_link, description, site_tags, tourism_type])

# دمج النموذج الأصلي مع البيانات المولدة
final_data = data + generated_data

# تحويلها لـ DataFrame وتصديرها كـ Excel
df = pd.DataFrame(final_data, columns=["Site Name", "Governorate", "Visiting Hours", "Foreigner Student Price", "Google Maps Link", "Description", "Tags", "Tourism Type"])
df.to_excel("tourism_data_50000.xlsx", index=False)
print("تم تصدير الملف بنجاح!")

ModuleNotFoundError: No module named 'openpyxl'

In [2]:
import pandas as pd
import random

# النموذج الأساسي (20 سطر كاملين)
data = [
    ["Salah El-Din Citadel", "Cairo", "5 AM - 8 PM", 60, "https://www.google.com/maps/search/?api=1&query=Salah+El-Din+Citadel", "A medieval Islamic-era fortress with mosques and museums.", "fortress|mosque|museum|medieval|cairo", "Historical"],
    ["Baron Palace", "Cairo", "9 AM - 5 PM", 60, "https://www.google.com/maps/search/?api=1&query=Baron+Palace", "A unique Hindu-style palace built in the early 20th century.", "palace|hindu|architecture|20th-century|cairo", "Cultural"],
    ["Islamic Art Museum", "Cairo", "9 AM - 5 PM", 6, "https://www.google.com/maps/search/?api=1&query=Islamic+Art+Museum", "A museum showcasing Islamic artifacts and architecture.", "museum|islamic|art|architecture|cairo", "Cultural"],
    ["Egyptian Museum", "Cairo", "9 AM - 5 PM", 36, "https://www.google.com/maps/search/?api=1&query=Egyptian+Museum", "Home to the world’s largest collection of ancient Egyptian artifacts.", "museum|ancient|egyptian|artifacts|cairo", "Historical"],
    ["Giza Pyramids", "Giza", "5 AM - 7 PM", 60, "https://www.google.com/maps/search/?api=1&query=Giza+Pyramids", "The iconic pyramids one of the Seven Wonders of the Ancient World.", "pyramids|ancient|wonder|giza|pharaohs", "Historical"],
    ["Sphinx", "Giza", "5 AM - 7 PM", "-", "https://www.google.com/maps/search/?api=1&query=Sphinx", "A limestone statue with a lion's body and a human head.", "statue|sphinx|ancient|giza|pharaohs", "Historical"],
    ["Karnak Temple", "Luxor", "5 AM - 6 PM", 60, "https://www.google.com/maps/search/?api=1&query=Karnak+Temple", "A vast temple complex dedicated to Amun-Ra in ancient Thebes.", "temple|ancient|amun-ra|luxor|pharaohs", "Historical"],
    ["Valley of the Kings", "Luxor", "5 AM - 5 PM", 60, "https://www.google.com/maps/search/?api=1&query=Valley+of+the+Kings", "The burial site of many ancient Egyptian pharaohs.", "tombs|pharaohs|ancient|luxor|burial", "Historical"],
    ["Abu Simbel Temple", "Aswan", "5 AM - 6 PM", 60, "https://www.google.com/maps/search/?api=1&query=Abu+Simbel+Temple", "A massive rock temple built by Pharaoh Ramses II.", "temple|rock-cut|ramses|aswan|ancient", "Historical"],
    ["Philae Temple", "Aswan", "7 AM - 5 PM", 60, "https://www.google.com/maps/search/?api=1&query=Philae+Temple", "An ancient temple complex dedicated to the goddess Isis.", "temple|isis|ancient|aswan|religious", "Religious"],
    ["Roman Theater", "Alexandria", "9 AM - 5 PM", 60, "https://www.google.com/maps/search/?api=1&query=Roman+Theater", "A well-preserved Greco-Roman amphitheater in Alexandria.", "theater|roman|greco|alexandria|ancient", "Historical"],
    ["Qaitbay Citadel", "Alexandria", "8 AM - 5 PM", 60, "https://www.google.com/maps/search/?api=1&query=Qaitbay+Citadel", "A 15th-century fortress built on the ruins of the Lighthouse of Alexandria.", "fortress|lighthouse|15th-century|alexandria", "Historical"],
    ["Al-Mawlawi Tekkiyeh", "Cairo", "9 AM - 5 PM", 60, "https://www.google.com/maps/search/?api=1&query=Al-Mawlawi+Tekkiyeh", "A historic Sufi lodge in Cairo associated with the Mevlevi order.", "sufi|dervish|lodge|cairo|cultural", "Cultural"],
    ["Ahmad Ibn Tulun Mosque", "Cairo", "8 AM - 5 PM", 60, "https://www.google.com/maps/search/?api=1&query=Ahmad+Ibn+Tulun+Mosque", "One of the oldest mosques in Egypt built in the 9th century.", "mosque|9th-century|spiral|cairo", "Religious"],
    ["Nileometer", "Cairo", "9 AM - 5 PM", 60, "https://www.google.com/maps/search/?api=1&query=Nileometer", "An ancient structure used to measure the Nile River’s flood levels.", "nile|ancient|measurement|cairo", "Historical"],
    ["Manial Palace", "Cairo", "9 AM - 5 PM", 60, "https://www.google.com/maps/search/?api=1&query=Manial+Palace", "A royal residence from the early 20th century with gardens and museums.", "palace|royal|20th-century|cairo", "Cultural"],
    ["The Hanging Church", "Cairo", "9 AM - 5 PM", 60, "https://www.google.com/maps/search/?api=1&query=The+Hanging+Church", "One of the oldest Coptic churches in Egypt built above a Roman fortress.", "church|coptic|ancient|cairo", "Religious"],
    ["Saqqara Necropolis", "Giza", "8 AM - 5 PM", 60, "https://www.google.com/maps/search/?api=1&query=Saqqara+Necropolis", "An ancient burial ground with pyramids and tombs.", "necropolis|pyramids|tombs|giza", "Historical"],
    ["Luxor Temple", "Luxor", "6 AM - 9 PM", 60, "https://www.google.com/maps/search/?api=1&query=Luxor+Temple", "An ancient temple in the heart of Luxor city.", "temple|ancient|luxor|pharaohs", "Historical"],
    ["Edfu Temple", "Aswan", "7 AM - 5 PM", 60, "https://www.google.com/maps/search/?api=1&query=Edfu+Temple", "A well-preserved Ptolemaic temple dedicated to Horus.", "temple|horus|ptolemaic|aswan", "Historical"],
]

# قوائم لتوليد بيانات عشوائية
governorates = ["Cairo", "Giza", "Luxor", "Aswan", "Alexandria", "Fayoum", "Minya", "Sohag", "Sinai"]
hours = ["5 AM - 8 PM", "9 AM - 5 PM", "7 AM - 5 PM", "8 AM - 5 PM", "6 AM - 9 PM"]
prices = [6, 36, 60, 100]
types = ["Historical", "Cultural", "Religious"]
tags = ["temple", "museum", "pyramid", "ancient", "fortress", "mosque", "church", "tombs", "pharaohs"]

# توليد 50,000 صف
generated_data = []
for i in range(50000):
    site_name = f"Site {i+1}"
    governorate = random.choice(governorates)
    visiting_hours = random.choice(hours)
    price = random.choice(prices)
    maps_link = f"https://www.google.com/maps/search/?api=1&query=Site+{i+1}"
    description = f"A unique site in {governorate} with historical significance."
    site_tags = "|".join(random.sample(tags, 3)) + f"|{governorate.lower()}"
    tourism_type = random.choice(types)
    generated_data.append([site_name, governorate, visiting_hours, price, maps_link, description, site_tags, tourism_type])

# دمج النموذج الأصلي مع البيانات المولدة
final_data = data + generated_data

# تحويلها لـ DataFrame وتصديرها كـ Excel
df = pd.DataFrame(final_data, columns=["Site Name", "Governorate", "Visiting Hours", "Foreigner Student Price", "Google Maps Link", "Description", "Tags", "Tourism Type"])
df.to_excel("tourism_data_50000.xlsx", index=False)
print("تم تصدير الملف بنجاح!")


تم تصدير الملف بنجاح!


In [4]:
pip install openpyxl


   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/250.9 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/250.9 kB 435.7 kB/s eta 0:00:01
   --------- ----------------------------- 61.4/250.9 kB 469.7 kB/s eta 0:00:01
   ----------- --------------------------- 71.7/250.9 kB 435.7 kB/s eta 0:00:01
   ------------------ ------------------- 122.9/250.9 kB 599.1 kB/s eta 0:00:01
   ------------------------------- ------ 204.8/250.9 kB 831.5 kB/s eta 0:00:01
   ----------------------------------- -- 235.5/250.9 kB 801.7 kB/s eta 0:00:01
   -------------------------------------  245.8/250.9 kB 718.0 kB/s eta 0:00:01
   -------------------------------------  245.8/250.9 kB 718.0 kB/s eta 0:00:01
   -------------------------------------- 250.9/250.9 kB 641.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\Domain\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import random

# النموذج الأساسي (20 سطر كاملين)
data = [
    ["Salah El-Din Citadel", "Cairo", "5 AM - 8 PM", 60, "https://www.google.com/maps/search/?api=1&query=Salah+El-Din+Citadel", "A medieval Islamic-era fortress with mosques and museums.", "fortress|mosque|museum|medieval|cairo", "Historical"],
    ["Baron Palace", "Cairo", "9 AM - 5 PM", 60, "https://www.google.com/maps/search/?api=1&query=Baron+Palace", "A unique Hindu-style palace built in the early 20th century.", "palace|hindu|architecture|20th-century|cairo", "Cultural"],
    ["Islamic Art Museum", "Cairo", "9 AM - 5 PM", 6, "https://www.google.com/maps/search/?api=1&query=Islamic+Art+Museum", "A museum showcasing Islamic artifacts and architecture.", "museum|islamic|art|architecture|cairo", "Cultural"],
    ["Egyptian Museum", "Cairo", "9 AM - 5 PM", 36, "https://www.google.com/maps/search/?api=1&query=Egyptian+Museum", "Home to the world’s largest collection of ancient Egyptian artifacts.", "museum|ancient|egyptian|artifacts|cairo", "Historical"],
    ["Giza Pyramids", "Giza", "5 AM - 7 PM", 60, "https://www.google.com/maps/search/?api=1&query=Giza+Pyramids", "The iconic pyramids one of the Seven Wonders of the Ancient World.", "pyramids|ancient|wonder|giza|pharaohs", "Historical"],
    ["Sphinx", "Giza", "5 AM - 7 PM", "-", "https://www.google.com/maps/search/?api=1&query=Sphinx", "A limestone statue with a lion's body and a human head.", "statue|sphinx|ancient|giza|pharaohs", "Historical"],
    ["Karnak Temple", "Luxor", "5 AM - 6 PM", 60, "https://www.google.com/maps/search/?api=1&query=Karnak+Temple", "A vast temple complex dedicated to Amun-Ra in ancient Thebes.", "temple|ancient|amun-ra|luxor|pharaohs", "Historical"],
    ["Valley of the Kings", "Luxor", "5 AM - 5 PM", 60, "https://www.google.com/maps/search/?api=1&query=Valley+of+the+Kings", "The burial site of many ancient Egyptian pharaohs.", "tombs|pharaohs|ancient|luxor|burial", "Historical"],
    ["Abu Simbel Temple", "Aswan", "5 AM - 6 PM", 60, "https://www.google.com/maps/search/?api=1&query=Abu+Simbel+Temple", "A massive rock temple built by Pharaoh Ramses II.", "temple|rock-cut|ramses|aswan|ancient", "Historical"],
    ["Philae Temple", "Aswan", "7 AM - 5 PM", 60, "https://www.google.com/maps/search/?api=1&query=Philae+Temple", "An ancient temple complex dedicated to the goddess Isis.", "temple|isis|ancient|aswan|religious", "Religious"],
    ["Roman Theater", "Alexandria", "9 AM - 5 PM", 60, "https://www.google.com/maps/search/?api=1&query=Roman+Theater", "A well-preserved Greco-Roman amphitheater in Alexandria.", "theater|roman|greco|alexandria|ancient", "Historical"],
    ["Qaitbay Citadel", "Alexandria", "8 AM - 5 PM", 60, "https://www.google.com/maps/search/?api=1&query=Qaitbay+Citadel", "A 15th-century fortress built on the ruins of the Lighthouse of Alexandria.", "fortress|lighthouse|15th-century|alexandria", "Historical"],
    ["Al-Mawlawi Tekkiyeh", "Cairo", "9 AM - 5 PM", 60, "https://www.google.com/maps/search/?api=1&query=Al-Mawlawi+Tekkiyeh", "A historic Sufi lodge in Cairo associated with the Mevlevi order.", "sufi|dervish|lodge|cairo|cultural", "Cultural"],
    ["Ahmad Ibn Tulun Mosque", "Cairo", "8 AM - 5 PM", 60, "https://www.google.com/maps/search/?api=1&query=Ahmad+Ibn+Tulun+Mosque", "One of the oldest mosques in Egypt built in the 9th century.", "mosque|9th-century|spiral|cairo", "Religious"],
    ["Nileometer", "Cairo", "9 AM - 5 PM", 60, "https://www.google.com/maps/search/?api=1&query=Nileometer", "An ancient structure used to measure the Nile River’s flood levels.", "nile|ancient|measurement|cairo", "Historical"],
    ["Manial Palace", "Cairo", "9 AM - 5 PM", 60, "https://www.google.com/maps/search/?api=1&query=Manial+Palace", "A royal residence from the early 20th century with gardens and museums.", "palace|royal|20th-century|cairo", "Cultural"],
    ["The Hanging Church", "Cairo", "9 AM - 5 PM", 60, "https://www.google.com/maps/search/?api=1&query=The+Hanging+Church", "One of the oldest Coptic churches in Egypt built above a Roman fortress.", "church|coptic|ancient|cairo", "Religious"],
    ["Saqqara Necropolis", "Giza", "8 AM - 5 PM", 60, "https://www.google.com/maps/search/?api=1&query=Saqqara+Necropolis", "An ancient burial ground with pyramids and tombs.", "necropolis|pyramids|tombs|giza", "Historical"],
    ["Luxor Temple", "Luxor", "6 AM - 9 PM", 60, "https://www.google.com/maps/search/?api=1&query=Luxor+Temple", "An ancient temple in the heart of Luxor city.", "temple|ancient|luxor|pharaohs", "Historical"],
    ["Edfu Temple", "Aswan", "7 AM - 5 PM", 60, "https://www.google.com/maps/search/?api=1&query=Edfu+Temple", "A well-preserved Ptolemaic temple dedicated to Horus.", "temple|horus|ptolemaic|aswan", "Historical"],
]

# قوائم لتوليد أسماء مصرية بالإنجليزي
prefixes = ["Temple of", "Citadel of", "Valley of", "Pyramid of", "Tomb of", "Church of", "Museum of", "Palace of", "Market of", "Mosque of"]
names = ["Amenhotep", "Ramses", "Nefertari", "Hatshepsut", "Tutankhamun", "Senusret", "Isis", "Horus", "Abu Bakr", "Salah El-Din", "Al-Mansour", "Nile", "Fatimid", "Coptic", "Luxorian", "Alexandrian", "Gizan", "Sohagian", "Fayoumi", "Sinai"]

# قوائم لتوليد بيانات عشوائية
governorates = ["Cairo", "Giza", "Luxor", "Aswan", "Alexandria", "Fayoum", "Minya", "Sohag", "Sinai"]
hours = ["5 AM - 8 PM", "9 AM - 5 PM", "7 AM - 5 PM", "8 AM - 5 PM", "6 AM - 9 PM"]
prices = [6, 36, 60, 100]
types = ["Historical", "Cultural", "Religious"]
tags = ["temple", "museum", "pyramid", "ancient", "fortress", "mosque", "church", "tombs", "pharaohs"]

# توليد 50,000 صف بأسماء مصرية بالإنجليزي
generated_data = []
for i in range(50000):
    site_prefix = random.choice(prefixes)
    site_name_part = random.choice(names)
    site_name = f"{site_prefix} {site_name_part}"  # مثال: "Temple of Horus" أو "Citadel of Salah El-Din"
    governorate = random.choice(governorates)
    visiting_hours = random.choice(hours)
    price = random.choice(prices)
    maps_link = f"https://www.google.com/maps/search/?api=1&query={site_name.replace(' ', '+')}"
    description = f"A unique {site_prefix.split(' of')[0].lower()} in {governorate} with historical significance."
    site_tags = "|".join(random.sample(tags, 3)) + f"|{governorate.lower()}"
    tourism_type = random.choice(types)
    generated_data.append([site_name, governorate, visiting_hours, price, maps_link, description, site_tags, tourism_type])

# دمج النموذج الأصلي مع البيانات المولدة
final_data = data + generated_data

# تحويلها لـ DataFrame وتصديرها كـ Excel
df = pd.DataFrame(final_data, columns=["Site Name", "Governorate", "Visiting Hours", "Foreigner Student Price", "Google Maps Link", "Description", "Tags", "Tourism Type"])
df.to_excel("tourism_data_50000.xlsx", index=False)
print("File exported successfully!")

File exported successfully!
